In [34]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import PyPDFLoader
from langchain.chains.question_answering import load_qa_chain
import os

In [35]:
os.environ['OPENAI_API_KEY'] = ''

In [36]:
# Load dos modelos
embeddings_model = OpenAIEmbeddings()
llm = ChatOpenAI(model='gpt-4o-mini', max_tokens=200)

In [37]:
# Carregar PDF
pdf_link = "/Users/rafaeltiburcio/dev/python/rag/pdf_example.pdf"
loader = PyPDFLoader(pdf_link, extract_images=False)
pages = loader.load_and_split()

In [38]:
# Separar pedaços de documentos
text_spliter = RecursiveCharacterTextSplitter(
    chunk_size=4000,
    chunk_overlap=20,
    length_function=len,
    add_start_index=True
)

chunks = text_spliter.split_documents(pages)

In [39]:
#Salvar no vector DB

db = Chroma.from_documents(chunks, embedding=embeddings_model, persist_directory='/Users/rafaeltiburcio/dev/python/rag/text_index')

In [40]:
# Carregar DB
vector_db = Chroma(persist_directory='/Users/rafaeltiburcio/dev/python/rag/text_index', embedding_function=embeddings_model)

# Load Retriever
retriever = vector_db.as_retriever(search_kwargs={"k": 3})

# construção da cadeia de prompt para chamada do LLM
chain = load_qa_chain(llm, chain_type="stuff")

In [41]:
def ask(question):
    context = retriever.get_relevant_documents(question)
    answer = (chain({ "input_documents": context, "question": question }, return_only_outputs=True))['output_text']
    return answer, context

In [42]:
user_question = input("User: ")
answer, context = ask(user_question)
print("Answer: ", answer)
print("Context", context)

Answer:  A recomendação para a lâmina C, conforme o relatório de inspeção, é a seguinte:

- Para o dano classificado como "Crack - Spanwise" (Dano 7), a reparação deve ser feita em até 6 meses, com a reparação a ser realizada "up-tower".
  
- Para o dano classificado como "Contamination" (Dano 8), é recomendado realizar uma re-inspeção na próxima interval de serviço.

Essas recomendações são categorizadas conforme a gravidade do dano.
Context [Document(metadata={'page': 0, 'source': '/Users/rafaeltiburcio/dev/python/rag/pdf_example.pdf', 'start_index': 0}, page_content='Classification: Confidential  \n \n \n  \n \n \n \n \n \n \n \n \n \n \n \n \nRenascenca I  \n46907'), Document(metadata={'page': 0, 'source': '/Users/rafaeltiburcio/dev/python/rag/pdf_example.pdf', 'start_index': 0}, page_content='Classification: Confidential  \n \n \n  \n \n \n \n \n \n \n \n \n \n \n \n \nRenascenca I  \n46907'), Document(metadata={'page': 11, 'source': '/Users/rafaeltiburcio/dev/python/rag/pdf_examp